# Decision Trees Exercise
In this exercise you will show that ID3 is sub-optimal. Implement a simple version of Decision Tree, and will then apply a Decision Tree classsifier on the MNIST hand written digits dataset that we already saw.


## Suboptimality of ID3
Consider the following training set, where $\mathcal{X} = \{0, 1\}^3$ and $\mathcal{Y} =\{0, 1\}$:

$$
\begin{aligned}
((1, 1, 1), 1)\\
((1, 0, 0), 1)\\
((1, 1, 0), 0)\\
((0, 0, 1), 0)
\end{aligned}
$$

Suppose we wish to use this training set in order to build a decision tree of depth 2 (i.e. for each
input we are allowed to ask two questions of the form "$x_i = 0$?" before deciding on the label).

1. Suppose we run the ID3 algorithm up to depth 2 (namely, we pick the root node and its
children according to the algorithm, but instead of keeping on with the recursion, we stop
and pick leaves according to the majority label in each subtree, once we reach depth 2). 
Assume that the subroutine used to measure the quality of each feature is based on the information gain, and that if two features get the same score, one of them is picked arbitrarily. 
Show that the training error of the resulting decision tree is at least 1/4.
2. Find a decision tree of depth 2, which attains zero training error.


#### Answer
Put your answer here...

## Implementing Decision Tree From Scratch
In this exercise you will need to implement a simple version of Decision Tree from scratch. Your decision tree will handle **continuous input and output** (this should actually work also for binary input attributes).

* Compelete the skeleton class below
  - `X` is a matrix of data values (rows are samples, columns are attributes)
  - `y` is a vector of corresponding target values
  - `min_leaf` is the minimal number of samples in each leaf node
  
* For splitting criterion, use either **"Train Squared Error Minimization (Reduction in Variance)"** or **"Train Absolute Error Minimization"** (choose one). Whatever you choose, make sure you implement the splitting point decision efficiently (in $O(nlgn)$ time).

* The `predict` function will use mean of the target values in the leaf node matching each row of the given `X`. The result is a vector of predictions matching the number of rows in `X`.

* To check your decision tree implementation, use the boston dataset (`from sklearn.datasets import load_boston`) split the data set into train and test using (`from sklearn.model_selection import train_test_split`)

  - Use the following to estimate what are the best hyper parameters to use for your model
```
    for min_leaf in [1,5,10,100]:
      dt = DecisionTree(X, y, n, sz, min_leaf)
      mse = # mean square error over test set
      print("min_leaf:{0} --- oob mse: {1}".format(min_leaf, mse))
```
  
  - Using your chosen hyperparameters as a final model, plot the predictions vs. true values of all the samples in the training set . Use something like:
  ```
  y_hat = dt.predict(X_train)  # forest is the chosen model
  plt.scatter(y_hat, y_test)
  ```

In [10]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston;


In [ ]:
class BinarySearchNode():
    def __init__(self, data):
        self.data = data 
        self.left = None 
        self.right = None
    
    def add_child(self, data, decision_boundary):
        self.decision_boundary = decision_boundary
        if data == self.data: #Need to check the value: If the value already exists, we don't continue-- we don't want duplicates 
            return

        if data < self.decision_boundary: #Condition: If new data is less than the parent node
            if self.left: #Check that our left element is not a leaf node
                self.left.add_child(data) #If it self.left has a child, add data as a child to self.left's child.
            else:
                self.left = BinarySearchNode(data) #If the left node is empty though, then it is already a leaf. We designate it as a node, with left and right = None.
        else:
            if self.right:
                self.right.add_child(data)
            else:
                self.right = BinarySearchNode(data)
    

    def in_order_traversal(self): #SORTING ALGORITHM
        elements = []
        #We sort from smallest to largest, so start with left tree
        if self.left:
            elements += self.left.in_order_traversal() #This iterates until we get to the bottom most left node, adds that node, then recursively adds the previous ones to our list.
        
        #Once we are done adding each left node, we have reached the root. Now, we add the root:
        elements.append(self.data)

        if self.right:
            elements += self.right.in_order_traversal()
        
        return elements

    
    

        

In [ ]:
def build_tree(rows):
    score, question = find_split(rows)

    if score == 0: return Leaf(rows)

    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows)

    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)
    

In [12]:
X, y = load_boston(return_X_y = True);
data = load_boston();
df = pd.DataFrame(data.data, columns = data.feature_names);
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [ ]:
class DecisionTree():
  def __init__(self, X, y, n_trees, sample_sz, min_leaf = 2, max_depth=2):
    self.root = None
    self.X = X
    self.y = y
    self.n_trees = n_trees
    self.sample_sz = sample_sz
    self.min_leaf = min_leaf
    self.max_depth = max_depth

  def build_tree(self, X, y, level = 0):
    num_samples, num_features = np.shape(dataframe)
    n_labels = np.unique(y)

    if level >= self.max_depth or n_labels == 1 or n_samples < self.min_leaf:
        leaf = self.most_common_label(y)

    features_idx = 
    best_feature, best_thresh = self.choose_best(X, y, )


  def predict(self, X):
    pass

In [105]:
class Make_Node():
    def __init__(self, feature_idx = None, threshold = None, left = None, right = None, variance = None, value = None):
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.left = left
        self.right = right
        self.variance = variance
        self.value = value #For leaf node

class RegressionTree():
    def __init__(self, max_depth, min_split):
        self.root = None
        self.min_split = min_split
        self.max_depth = max_depth


    def split(self, df, feature_idx, threshold):
        left_child = np.array([sample for sample in df if sample[feature_idx] <= threshold])
        right_child = np.array([sample for sample in df if sample[feature_idx] > threshold])
        return left_child, right_child

    def train_squared_error(self, y, y_left, y_right):

        weight_left = len(y_left) / len(y)
        weight_right = len(y_right)/ len(y)
        variance = weight_left * np.var(y_left) + weight_right * np.var(y_right)

        #code says to subtract the parents error by the error fo the children. Not sure why we aren't just returning the variance of the children.
        return variance 



    def get_leaf(self, y):
        return np.mean(y) #presentation says to take the mode?

    def best_feature(self, df, n_samples, n_features):
        best_split_dict = {}
        max_reduction = np.inf #Change variable to max_trained_squared_error?

        for feat_idx in range(n_features):
            feat_vals = df[:, feat_idx]
            threshold_options = np.unique(feat_vals)

            for threshold in threshold_options:
                left_child, right_child = self.split(df, feat_idx, threshold)
            
                if len(left_child) > 0 and len(right_child) > 0:
                    y, left_y, right_y = df[:, -1], left_child[:, -1], right_child[:, -1]

                    trained_squared_error = self.train_squared_error(y, left_y, right_y)

                    if trained_squared_error < max_reduction: #We want minimum variance. (If the trained_error function were to subtract frm the variance of the parent, flip to greater than).
                            best_split_dict["feature_idx"] = feat_idx
                            best_split_dict["threshold"] = threshold
                            best_split_dict["left_child"] = left_child
                            best_split_dict["right_child"] = right_child
                            best_split_dict["variance"] = trained_squared_error
                            max_reduction = trained_squared_error
        
        return best_split_dict


    def fit(self, X, y):
        df = np.concatenate((X, y), axis =1)
        self.root = self.build_tree(df)

    def build_tree(self, df, level = 0):
        X, y = df[:, :-1], df[:,-1]
        n_samples, n_features = np.shape(X)
        best_split = {} #hold all features of the split in  dict.
        if n_samples>= self.min_split and level <=self.max_depth:    #a1) Find the best feature for our tree to split on, if we haven't reached past our required tree size.

            best_split = self.best_feature(df, n_samples, n_features)

            if best_split['variance'] > 0:                          #a2) Split the tree based off this best feature into left and right subtrees
                left_tree = self.build_tree(best_split['left_child'], level +1)
                right_tree = self.build_tree(best_split['right_child'], level+1)

                return Make_Node(best_split['feature_idx'], best_split['threshold'], left_tree, right_tree, best_split['variance']) #a3) Make current split into Parent Node
        
        leaf = self.get_leaf(y)                                      #b1) if tree size has already been reached, (or reduction = 0), then create and return a Leaf Node.

        return Make_Node(value = leaf)


    def print_tree(self, columns, tree=None, indent=" ", ):
            ''' function to print the tree '''
            self.columns = columns

            if not tree:
                tree = self.root

            if tree.value is not None:
                print(tree.value)

            else:
                # if self.columns is None:
                #     print("X_"+str(tree.feature_idx), "<=", tree.threshold, "?", tree.variance)
                    
                # else:
                print("X_"+str(self.columns[tree.feature_idx]), "<=", tree.threshold, "?", tree.variance)

                print("%sleft:" % (indent), end="")
                self.print_tree(tree.left, indent + indent)
                print("%sright:" % (indent), end="")
                self.print_tree(tree.right, indent + indent)


In [101]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [106]:
regressor = RegressionTree(min_split=2, max_depth=2)
regressor.fit(X_train, Y_train)
regressor.print_tree(columns= df.columns)

X_INDUS <= 7.38 ? 32.287344445753966
 left:

AttributeError: 'str' object has no attribute 'value'

In [97]:
columns = len(None)

if columns:
    print('yes')

TypeError: object of type 'NoneType' has no len()

## Using Decision Treefor Digits Classification
Remeber the MNIST dataset used - you will now test the power of decision trees on this problem.
This time you are given a free hand in choosing the test and train set sizes, model parameters (such as gain function and constraints over the trees) and features (whether to use binary pixel values or the original continous gray value).
- Choose which model parameters you wish to optimize, explain how would you do that, and find a model which you believe would have the minimal generalization error --- do this for both a single decision tree model, and a random forest.
  - You can use `sklearn.tree.DecisionTreeClassifier`
- Once you are satisfied with the model parameters, plot the importance of each of the pixels to the final decision.
- Last, estimate the class assignment probabilities for all the correctly classified and misclassified examples in your test data. (confusion matrix)
- Discuss your results.

In [ ]:
# code and answer go here